# Snow Density Calculations

This notebook demonstrates methods for calculating density from common snow pit measurements (hand hardness and grain form) using both the local snowpyt_mechparams package and snowpylot for CAAML parsing.


In [2]:
# Import Libraries
import os
import xml.etree.ElementTree as ET
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch
import glob

# Import snowpylot for CAAML parsing
from snowpylot import caaml_parser

# Add the src directory to the path to import snowpyt_mechparams
sys.path.append('../src')
from snowpyt_mechparams import density


Parse Snowpit Files

In [3]:
all_pits = []

folder_path = 'data'

for file in os.listdir(folder_path):
    file_path = folder_path + "/" + file  # create the file path
    all_pits.append(caaml_parser(file_path))


In [4]:
# Collect relevant data from each snowpit

pit_info = []
layer_info = []

for pit in all_pits:
    pit_dict = {
        'pit_id': pit.core_info.pit_id,
        'layer_count': len(pit.snow_profile.layers),
    }
    pit_info.append(pit_dict)

    for layer in pit.snow_profile.layers:
        layer_dict = {
            'pit_id': pit.core_info.pit_id,
            'depth_top': layer.depth_top,
            'thickness': layer.thickness,
            'hand_hardness': layer.hardness,
            'wetness': layer.wetness,
            'layer_of_concern': layer.layer_of_concern,
            'grain_form_primary': layer.grain_form_primary,
            'grain_form_secondary': layer.grain_form_secondary,
        }
        layer_info.append(layer_dict)

# Create a dataframe from the pit and layer info
pit_df = pd.DataFrame(pit_info)
layer_df = pd.DataFrame(layer_info)


In [ ]:
# Dataset Summary
print("=== Dataset Summary ===")
print(f"Total number of snowpits: {len(all_pits)}")
print(f"Total number of layers: {len(layer_df)}")

# Filter layers with both hand hardness and grain form data
layers_with_data = layer_df[layer_df['hand_hardness'].notna() & layer_df['grain_form_primary'].notna()]
print(f"Layers with both hand hardness and grain form data: {len(layers_with_data)}")
print(f"Percentage of layers with complete data: {len(layers_with_data)/len(layer_df)*100:.1f}%")


=== Dataset Summary ===
Total number of snowpits: 43829
Total number of layers: 324054
Layers with both hand hardness and grain form data: 244247
Percentage of layers with complete data: 75.4%


Implement Geldsetzer Method

In [9]:
# Calculate density using Geldsetzer method for each layer
def calculate_layer_density(row):
    try:
        return density.calculate_density(
            method='geldsetzer',
            hand_hardness=row['hand_hardness'],
            grain_form=row['grain_form_primary']
        )
    except ValueError as e:
        # Return NaN for cases where density cannot be calculated
        return np.nan

# Add density column to the dataframe
layers_with_data = layers_with_data.copy()  # Avoid SettingWithCopyWarning
layers_with_data['density'] = layers_with_data.apply(calculate_layer_density, axis=1)

# Show summary of calculated densities
print("=== Density Calculation Results ===")
print(f"Layers with calculated density: {layers_with_data['density'].notna().sum()}")
print(f"Layers where density could not be calculated: {layers_with_data['density'].isna().sum()}")
print(f"Success rate: {layers_with_data['density'].notna().sum()/len(layers_with_data)*100:.1f}%")

# Add density to the dataframe
layers_with_data['density'] = layers_with_data['hand_hardness'].apply(density.geldsetzer)



=== Density Calculation Results ===
Layers with calculated density: 0
Layers where density could not be calculated: 244247
Success rate: 0.0%


AttributeError: module 'snowpyt_mechparams.density' has no attribute 'geldsetzer'